<a href="https://colab.research.google.com/github/xjdeng/textual_inversion_colab/blob/main/textual_inversion_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ### Config
from google.colab import drive

mount_drive = True

# @markdown ## Most people will want to change these

ti_name = "" # @param {"type":"string","placeholder":"(REQUIRED)"}
class_name = "*" # @param {"type":"string"}
vectors = 5 # @param {"type":"integer"}
image_dir = "/content/textual_inversion_colab/dataset" # @param {"type":"string"}
output_dir = "/content/drive/MyDrive/ti_output" # @param {"type":"string"}



# @markdown ## Some people may want to change these


resize_images = True #@param {type: "boolean"}
epochs = 1400 # @param {"type":"integer"}
save_every_n_epochs = 700 # @param {"type":"integer"}
batch_size = 6 # @param {"type":"integer"}
learning_rate = 5.0e-04 # @param {"type":"number"}

# @markdown ## Most people can safely ignore these

gradient_accumulation = 1 # @param {"type":"integer"}
huggingface_repo = "xjdeng/stable-diffusion-1.5" # @param {"type":"string"}

if mount_drive:
  drive.mount('/content/drive')


In [ ]:
#@title ### Installation and Setup
!git clone https://github.com/xjdeng/textual_inversion_colab
!git clone https://github.com/huggingface/diffusers
%cd diffusers
!pip install .
!pip install -r /content/diffusers/examples/textual_inversion/requirements.txt
!pip install -r /content/textual_inversion_colab/requirements.txt
!pip install -U xformers torchvision --index-url https://download.pytorch.org/whl/cu121
%cd ../
from path import Path as path
if resize_images:
  !git clone https://github.com/xjdeng/sd_tools
  from sd_tools import imagetools
  import cv2
  DATA_DIR = f"{image_dir}/resized"
  path(DATA_DIR).mkdir_p()
  for f in path(image_dir).files():
    f.copy(DATA_DIR)
  for f in path(DATA_DIR).files():
    img = cv2.imread(f)
    if img is None:
      continue
    resized = imagetools.resize(img)
    cv2.imwrite(f, resized)
else:
  DATA_DIR = image_dir

path(output_dir).mkdir_p()
%cd /content/textual_inversion_colab

In [ ]:
#@title ### Training
#MODEL_NAME="pt-sk/stable-diffusion-1.5"
placeholder = '"<' + ti_name + '>"'
initializer = f'"{class_name}"'
prompt = f'"{ti_name}"'
exception = False
model = f'"{huggingface_repo}"'
try:
 !accelerate launch textual_inversion.py \
    --pretrained_model_name_or_path=$model \
    --train_data_dir=$DATA_DIR \
    --num_vectors=$vectors \
    --learnable_property="object" \
    --placeholder_token=$placeholder\
    --initializer_token=$initializer \
    --resolution=512 \
    --train_batch_size=$batch_size \
    --gradient_accumulation_steps=$gradient_accumulation \
    --max_train_steps=$epochs \
    --checkpointing_steps=$save_every_n_epochs \
    --learning_rate=$learning_rate \
    --scale_lr \
    --lr_scheduler="constant" \
    --lr_warmup_steps=0 \
    --output_dir=$output_dir \
    --enable_xformers_memory_efficient_attention \
    --num_validation_images 1 \
    --validation_prompt=$prompt\
    --resume_from_checkpoint="latest"
except Exception as e:
  exception = True


In [ ]:
#@title ### Shutdown
import time
if not exception:
  rm1 = f"{output_dir}/checkpoint*"
  !rm -rf $rm1
  !rm -rf $rm2
time.sleep(300)
from google.colab import runtime
runtime.unassign()